# Personal Search 01

Busca de informações em livros usando FAISS

## Import packages

In [1]:
from langchain_community.llms import Ollama
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser
from operator import itemgetter
from langchain_community.llms import Ollama
from IPython.display import display, Markdown
import logging

#logging.basicConfig(level=logging.INFO)

## Configuration

In [2]:
model = "mistral:7b"
embeddings_model = "nomic-embed-text:v1.5"
path_books = "./vector-store/books"
question = "Relacionamento com Deus"

## Retriever Instance

In [3]:
embeddings = OllamaEmbeddings(model=embeddings_model)

db = FAISS.load_local(path_books, embeddings, allow_dangerous_deserialization=True)

retriever = db.as_retriever()
retriever.search_kwargs["distance_metric"] = "cos"
retriever.search_kwargs["fetch_k"] = 100
retriever.search_kwargs["maximal_marginal_relevance"] = False
retriever.search_kwargs["k"] = 5

## Search

In [10]:
# RAG
template = """
Você é um assistente de busca de conhecimento.
Traga citações diretas do contexto ou o que mais se aproxima do assunto "{question}". 

Contexto:
{context} 

Resposta:
"""

prompt = ChatPromptTemplate.from_template(template)

llm = Ollama(model=model, temperature=0)

final_rag_chain = (
    {"context": retriever, 
     "question": itemgetter("question")} 
    | prompt
    | llm
    | StrOutputParser()
)

final_answer = final_rag_chain.invoke({"question":question})

In [11]:
display(Markdown(f"**Busca:** {question}\n\n"))
display(Markdown(final_answer))

**Busca:** Relacionamento com Deus



 Este texto contém duas partes distintas, cada uma delas escrita por um autor diferente e tratando de temas distintos, embora haja algumas semelhanças temáticas. A primeira parte é do livro "Conselhos para obreiros" de Charles H. Spurgeon e trata sobre a importância de manter uma consciência pura e servir a Deus sem compromissos, mesmo que isso significue sofrer perseguição ou humilhações. A segunda parte é do livro "Evangelho Maltrapilho" de Brennan Manning e trata sobre a importância de manter uma mente aberta e abraçar a graça divina sem se preocupar excessivamente com estruturas religiosas ou ideologias.

Ambos os textos enfatizam a importância de permanecer fiel às verdades básicas da fé cristã e evitar caírem em hipocrisia ou se encerrarem em dogmas rigorosos que podem limitar a relação com Deus. Eles também enfatizam a importância de ser amigos com pessoas diferentes, incluindo aqueles de outras denominações e religiões, e evitar se enquadrar excessivamente em ideologias ou estruturas que podem limitar a compreensão da graça divina.

A história sobre as duas comunidades mencionadas no final do texto de Manning serve para ilustrar esses princípios, mostrando como uma comunidade de alcoólicos e outra comunidade de Roslyn podem ensinar aos leitores a importância de manter uma mente aberta e abraçar a graça divina sem se preocupar excessivamente com estruturas religiosas ou ideologias.

In [24]:
sources = retriever.invoke(question)
for source in sources:
    display(Markdown(f"**Fonte:** {source.metadata}"))
    display(Markdown(f"**Trecho:** {source.page_content}"))

**Fonte:** {'source': './books/import/Evangelho Maltrapilho - Brennan Manning.pdf', 'page': 102}

**Trecho:** ~ 103 ~ misericordioso para conosco. Não permitiremos que esses ataques nos levem aos pecados da 
autopreocupação e da autodefesa. Nadando no am or misericordioso do Cristo redentor, somos 
livres para rir da tendência humana de assumir ares de superioridade espiritual — até mesmo em 
nós mesmos. Somos livres para estender aos outros a misericórdia que recebemos. 
Impregnado em tamanha misericórdia, o maltra pilho encontra depressa uma desculpa para 
isentar o vizinho que acordou com o pé esquerdo, e um modo de esquivar-se em vez de retaliar 
quando um dardo de ira é lançado em sua dire ção. "Suportai-vos uns aos outros, perdoai-vos 
mutuamente, caso alguém tenha motivo de quei xa contra outrem", ele lembra. "Assim como o 
Senhor vos perdoou, assim também perdoai vós; ac ima de tudo isto, porém, esteja o amor, que 
é o vínculo da perfeição" (Cl 3:13-15). O seu exem plo é o de Jesus, que ao morrer, humilhado e 
injustamente vilipendiado, clamou em favor de se us assassinos: "Pai, pe rdoa-lhes, porque não 
sabem o que fazem" (Lc 23:34). 
O Grande Livro dos Alcoólicos Anônimos ofer ece um adorável fragmento de sabedoria 
caseira para aquelas ocasiões em que somos confrontados com a injustiça de outros que 
tentaram deliberadamente (e talvez tenham conseguido) nos prejudicar: 
Essa foi a nossa postura. Percebemos qu e as pessoas que nos prejudicaram estavam 
talvez espiritualmente doentes. Embora não gostássemos dos sintomas e do modo 
como eles nos afetavam, elas, como nós, estavam também doentes. Pedimos a Deus 
que nos ajudasse a mostrar a elas a mesm a tolerância, compaixão e paciência que 
mostraríamos com toda a disposição a um amigo doente. Quando uma pessoa ofendia, 
dizíamos a nós mesmos: "Eis uma pessoa doente. De que forma posso ajudá-la? Deus, salva-me de perder a paciência. Seja feita a tua vontade".
104 
 Quando você já fez da sua vida algo tão ruim  que tomou-se irreconhecível, como já fizeram 
muitos de nós alcoólicos em recuperação, a compaixão prova-se um pouquinho mais fácil se 
você mostrar-se escrupuloso o bastante para le var em conta o seu próprio inventário em vez do 
da outra pessoa. 
Com um Deus cujo nome e Misericórdia, não é de admirar que a gratidão ultrapasse 
qualquer outra atitude ao longo de nossa jornada espiritual. Não passamos pela vida alheios ao 
preço pago pela nossa libertação. Uma reflexão ponderada traz à memória numerosos resgates e

**Fonte:** {'source': './books/import/Evangelho Maltrapilho - Brennan Manning.pdf', 'page': 64}

**Trecho:** ~ 65 ~ gostaria? O seguinte cenário provoca minha im aginação: Uma mulher humilde me procura por 
causa da minha exaltada reputação como guia espiritual. Ela é simples e direta: 
— Por favor, ensine-me a orar. 
— Fale-me sobre sua vida de oração — pergun to, sucintamente. Ela baixa os olhos e diz 
contritamente: — Não há muito que dizer. Eu oro antes da s refeições. Arrogantemente, eu replico: 
— Ora antes das refeições? Mas isso não é dema is, minha senhora? Eu  oro quando acordo 
e antes de me recolher, e oro novamente antes de le r o jornal e de ligar a televisão. Oro antes de 
caminhar e de meditar, antes do teatro e da ópera, antes de corre r, antes de nadar, de andar de 
bicicleta, de jantar, de palestrar e de  escrever. Oro até mesmo antes de orar. 
Naquela noite, transbordando de auto-aprovação, vou à presença do Senhor. E ele sussurra: 
— Seu joão-ninguém ingrato. Até o desejo de orar é dom meu. Há uma antiga lenda cristã que conta: "Qua ndo o Filho de Deus foi pregado à cruz e 
entregou o espírito, foi diretamente da cruz ao  inferno para libertar todos os pecadores que 
estavam sendo atormentados ali. O Diabo entã o chorou e lamentou porque achava que não 
conseguiria mais pecadores para o inferno. Deus  então disse a ele: 'Não chore, porque vou 
mandar até você toda aquela gente santa que se  tornou complacente com a consciência da sua 
bondade e cheia de justiça própria na sua condenaç ão dos pecadores. E o inferno ficará repleto 
mais uma vez por gerações, at é que eu venha novamente".
66 
Quanto tempo será necessário até que descubramos que não somos capazes de ofuscar a 
Deus com nossas realizações? 
Quando reconheceremos que não precisamos e não temos como comprar o favor de Deus? 
Quando reconheceremos que não temos de for ma alguma os requisitos necessários e 
aceitaremos alegremente o dom da graça? Quan do iremos apreender a empolgante verdade de 
Paulo: "sabendo, contudo, que o homem não é just ificado por obras da lei, e sim mediante a fé 
em Cristo Jesus" (Gl 2:16)? 
A fé autêntica nos leva a tratar os outros co m seriedade incondicional e a uma reverência 
amorosa pelo mistério da personalidade humana . O cristianismo autêntico deveria conduzir à 
maturidade, à personalidade e à realidade. De veria moldar homens e mulheres que vivam 
integralmente vidas de amor e de comunhão. 
A religião falsa e manipulada produz o efeito oposto. Quando a religião demonstra desdém

**Fonte:** {'source': './books/import/Evangelho Maltrapilho - Brennan Manning.pdf', 'page': 85}

**Trecho:** Em algum ponto da nossa vida, fomos fund amentalmente tocados por um encontro 
profundo com Jesus Cristo. Foi uma experiên cia de apogeu, um momento de imensa 
consolação. Fomos tomados de paz, alegria, confiança, amor. Dito simplesmente, fomos 
arrebatados. Nossa mente e coração ress oavam com maravilha e assombro. Ficamos 
profundamente comovidos por algumas horas, dias ou semanas, e eventualmente retornamos às ocupações rotineiras da existência diária. Não permanecemos resolvidos. 
Devagar ficamos emaranhados nas exigências do ministério ou da carreira e nas distrações 
que nosso agitado mundo oferece. Começamos a tratar Jesus como um velho amigo do 
Brooklyn que amamos intensamente em anos  passados mas com o qual gradualmente 
perdemos contato. Isso, é cla ro, não foi intencional. Simp lesmente permitimos que as 
circunstâncias nos separassem. Numa recente visita àquela cidade nunca passou pela nossa cabeça contatá-lo. Ficamos ocupados com alguma  outra coisa, mesmo que tenha se mostrado 
muito menos estimulante e cativante. É possíve l que nunca tenhamos amado tanto outra pessoa 
quanto amamos esse amigo, mas até me smo a lembrança tornou-se indistinta. 
Acentuado pelo agnosticismo da negligência — a falta de disciplina pessoal com relação ao 
bombardeio da mídia, o controle da mente, as conversas estéreis, a oração pessoal e a sujeição 
dos sentidos —, a presença de Jesus torna-se cada vez mais remota. Da mesma forma que a falta 
de dedicação e atenção dissolve a confiança e a comunhão nos relacionamentos humanos, a 
negligência do Espírito desfaz o tecido do relacionamento divi no. À medida que, olhando para 
                                                             
 
88 VICTOR RANGEL‐RIBEIRO.  Readers Digest, AUGUST 1989, P. 76.

**Fonte:** {'source': './books/import/Conselhos para obreiros - Charles H Spurgeon.pdf', 'page': 55}

**Trecho:** 12
Na escola dominical
rabalho na escola dominical é beneficência. E como poderia ser de outra forma, se é um
ato de obediência? Creio que você começou a fazer esse trabalho porque chama Jesus
de seu Mestre e Senhor e deseja cumprir o grande mandamento: “Ide por todo o mundo,
e pregai o evangelho a toda criatura” (Mc 16.15). Você vê as crianças como criaturas, criaturas
decaídas, mas ainda coisinhas adoráveis, cheias de vigor, vida e alegria. Você as vê como parte
da humanidade e conclui, imediatamente, que a ordem de seu Mestre se aplica a elas. Você não é
como os discípulos que queriam afastá-las, pois aprendeu com o erro deles e se lembra das
palavras do Mestre: “Deixai vir a mim as crianças e não as impeçais” (Mt 19.14). Você sabe,
também, que ele conferiu força à boca de bebês e crianças de peito, por causa do inimigo; de
modo que tem certeza de que ele incluiu os pequeninos na Grande Comissão, quando disse:
“Pregai o evangelho a toda criatura” (Mc 16.15).T
Você está duplamente convicto de estar obedecendo à vontade dele, porque há certos preceitos
especiais que se relacionam com os pequenos, como “cuida dos meus cordeiros” (Jo 21.15) e
“Instrui a criança no caminho em que deve andar, e mesmo quando envelhecer não se desviará
dele” (Pv 22.6). Você sabe que é nosso dever manter vivo o testemunho no mundo, portanto está
ansioso por ensinar a Palavra a seus filhos para que eles possam ensiná-la aos filhos deles, e
assim, de geração em geração, a Palavra do Senhor possa ser conhecida. Seja a tarefa agradável a
você, seja cansativa, não deve hesitar, mas, sim, obedecer. O amor que o redimiu também o
constrange. Você sente o toque da mão sagrada em seu ombro, a mão que foi transpassada, e
ouve seu Redentor dizer: “Assim como o Pai me enviou, também eu vos envio” (Jo 20.21), e, por
causa desse envio, você vai aos pequeninos em obediência à vontade do Senhor. Quem obedece
está agindo bem, e, nesse sentido, o seu serviço entre os pequeninos é beneficência.
É beneficência, também, porque  traz glória a Deus . Devemos sempre continuar a receber de
Deus, que é a grande fonte de bondade e bênção, mas ainda assim, em infinita condescendência,
ele nos permite dar-lhe algum retorno. Assim como a gota de orvalho reflete o raio com que o
grande Sol a adorna, possamos nós também, dentro de nossa limitação, fazer a luz do nosso
grande Pai brilhar diante dos olhos dos homens. Nossa vida pode ser como os rios que correm

**Fonte:** {'source': './books/import/Evangelho Maltrapilho - Brennan Manning.pdf', 'page': 52}

**Trecho:** ~ 53 ~ pois, porque não sabeis quando virá o dono da casa : se à tarde, se à meia-noite, se ao cantar do 
galo, se pela manhã; para que, vindo ele in esperadamente, não vos ache dormindo. O que, 
porém, vos digo, digo a todos: vigiai!" 
Na parábola do banquete de casamento, o co nvidado que não está trajando roupa de festa 
é arrastado à força pelos leões-de-chácara e arre messado porta afora. "A roupa de festa é o 
arrependimento. Revistam-se com ele hoje, ante s da sua morte, no di a antes que o dilúvio 
comece. Vistam-no hoje! A exigência da crise é a conversão."52 
Roma está em chamas, diz Je sus. Largue a vadiagem, mude  de vida e venha até mim. 
Quando um tornado vem devastando rua acima não é hora de parar para ch eirar as flores. Abra 
mão dos velhos tempos que nunca existiram — uma igreja organizada que você nunca 
freqüentou, virtudes tradicionais que você nunc a praticou, obediência legalista que você nunca 
observou e a ortodoxia estéril que você nunca aceitou. A velha era já passou. A decisiva 
intervenção de Deus já aconteceu. 
A pessoa que percebe a gravidade da situação  sabe que a decisão não permite qualquer 
adiamento. O Contador de Hist órias nos chama não ao medo, mas à ação. Não se apegue a 
bijuterias baratas quando a pérola de grande  valor está sendo oferecida. Quando a própria 
existência de uma pessoa está sendo ameaçada, quando ele ou ela estão no limiar da ruína 
m o r a l ,  q u a n d o  e s t á  t u d o  e m  j o g o ,  a  h o r a  c h egou para um momento de decisão ousado e 
resoluto. 
Na parábola dos talentos, os três servos sã o chamados para prestar contas do modo que 
usaram os dons confiados a eles. Os dois primeir os utilizaram seus talentos de modo ousado e 
empreendedor. O terceiro, que prudentemente em bala seu dinheiro e o enterra, tipifica o 
cristão que deposita sua fé num re cipiente hermeticamente fechado e sela a tampa. Ele ou ela 
manqueja pela vida com base nas lembranças da Escola Dominical e recusa teimosamente o 
desafio do crescimento e da ma turidade espiritual. Não dispos ta a correr riscos, essa pessoa 
perde o talento confiado a ela. "O senhor queria  que seus servos assumissem riscos. Ele queria 
que eles apostassem com o seu dinheiro".53 
Numa parábola manifestamente po lêmica para sua audiência juda ica, Jesus conta a história 
de um administrador ardiloso. Somos apresentad os a um defraudador qu e, para cobrir suas